# 进程管理  
1. 进程创建  
2. 进程调度  
3. 进程终止  
4. 僵尸进程  
5. 子进程异步清除  
6. 守护进程  
---   

## 进程创建   
**使用system()方法创建进程**  
```C++  
    //*******
    //use system() to create a process
    //*******
    int ret_val = system("ls -la /");
    cout << ret_val << endl;
```  

**使用fork()方法创建进程**  
使用`fork()`方法创建的子进程会继承父进程的工作目录、文件掩码、全部文件描述符等内容。（相当于拷贝了一份父进程成子进程）。  

直接运行程序，将输出两次`Done!`，因为父进程和子进程各输出一次。  
多进程的调试是一个问题，直接运行时不会输出在子进程中输出的内容。单步调试则可以进入子进程，但是同样不会输出父进程中的内容。
```C++  
void my_createprocess_main(){

    //*******
    //use fork() to create a new process which is inherit from the parent process
    //*******
    cout << "the parent peocess ID is:" << (int) getpid() << endl;
    pid_t child_pid = fork();
    if(child_pid != 0){
        cout << "this is the parent process, with id:" << (int)getpid() << endl;
        cout << "the child process ID is:" << (int)child_pid << endl;
    }else{
        cout << "this is the child process ,with id:" << (int)getpid() << endl;
    }

    //use exec() to execute something in the child process
    char* args[] = {"ls", "-la", "/", NULL};
    spwan("ls", args);
    cout << "Done! \n";
}

int spwan(char* program, char** args){
    pid_t child_pid = fork();
    if(child_pid != 0){
        return child_pid;
    }else{
        execvp(program, args);
        //when something bad happend, the execvp function will have returned value
        cerr << "Error occurred when executing execvp !\n";
        abort();
    }
}
```  

## 进程调度  
**进程调度策略、处理器亲和性**的一系列函数在`sche.h`头文件中  
**进程优先级调整**的一系列函数在`sys/time.h`头文件中  
> 处理器亲和性：通过代码选择执行的处理器核  

## 进程终止  
**终止进程函数：kill**  
> kill 命令通常用来 “杀死”（终止）进程，它可以用来终止运行不正常的程序或者反过来拒绝终止的程序。  
  kill 命令准确地说并不是 “杀死” 进程，而是给进程发送信号（signal）。

**等待进程结束函数：wait()、watipid()**  
> wait()一般用于等待子进程结束  
  waitpid()用于等待特定pid的进程结束  
  
## 僵尸进程  
一般子进程执行结束后需要被父进程清除。  
> 这里的**清除**指的是在父进程中使用wati方法等待子进程结束，然后将子进程的状态取出，之后子进程就会被清除。  
  进程一旦调用了wait就立即阻塞自己，由wait自动分析是否当前进程的某个子进程已经退出。  
  1. 如果让它找到了这样一个已经变成僵尸的子进程，wait就会收集这个子进程的信息，并把它彻底销毁后返回；  
  2. 如果没有找到这样一个子进程，wait就会一直阻塞在这里，直到有一个出现为止。

但如果父进程没有等待子进程，则子进程的状态没有被取出，然后成为了僵尸进程，子进程中的资源也不会被回收。  
**最后未清除的子进程会自动被init进程收养**  

## 子进程异步清除  
为了不产生僵尸进程，父进程要等子进程执行结束后（使用wait方法）对其进行回收。但是这种方法会造成**父进程的阻塞**，使用子进程异步清除解决这个问题。  
**在子进程执行结束后，会自动向父进程发送SIGCHLD信号**，父进程只要写一个对SIGCHLD信号的处理函数即可。  

问题：  
> 多进程程序的调试还不清楚。

## 守护进程  
先记住。  
创建守护进程的步骤：  
1. 创建新进程  
2. 守护进程的父进程退出  
3. 守护进程创建新进程组和新会话  
4. 改变工作目录  
5. 重设文件权限掩码  
6. 关闭所有文件描述符  
7. 标准流重定向到/dev/null  

使用`daemon()`方法实现之前的7个步骤  

